<a href="https://colab.research.google.com/github/HCboy01/DL_Project/blob/main/Kim/DL_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1.라이브러리 설치 및 임포트

In [2]:
# PyTorch 및 기타 라이브러리 설치
!pip install torch torchvision matplotlib

import os
import zipfile
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


2. 데이터 준비

In [3]:
from torch.utils.data import random_split, DataLoader

# 데이터 경로 설정
train_dir = "/content/drive/MyDrive/study/finalproject_dataset/train"
test_dir = "/content/drive/MyDrive/study/finalproject_dataset/test"

# 데이터 증강 변환 정의
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomResizedCrop(224),        # 임의로 크롭 후 리사이즈
    transforms.RandomHorizontalFlip(p=0.5),  # 50% 확률로 좌우 뒤집기
    transforms.RandomRotation(20),           # 임의의 각도로 회전 (-20도 ~ 20도)
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # 색상 변화
    transforms.ToTensor(),                   # 텐서로 변환
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # 정규화
])

# 테스트 데이터 변환 (증강 없이 기본적인 변환만)
val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),           # 크기 조정
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [4]:
# Train 데이터셋 로드
dataset = datasets.ImageFolder(root=train_dir)
test_dataset = datasets.ImageFolder(root=test_dir, transform=val_test_transform)

# Train-Validation Split 비율 설정
train_size = int(0.8 * len(dataset))  # 80%를 Train으로
val_size = len(dataset) - train_size  # 나머지 20%를 Validation으로

# Split
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataset.dataset.transform = train_transform  # Train 데이터 증강
val_dataset.dataset.transform = val_test_transform      # Validation 데이터 원본 유지
# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
print(f"Train Data: {len(train_dataset)}, Validation Data: {len(val_dataset)}")


Train Data: 248, Validation Data: 62


3. 데이터 전처리 및 로더

4. 모델 정의 (Pretrained ResNet 사용)

In [5]:
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample  # 입력 차원이 맞지 않을 때 차원 조정용

    def forward(self, x):
        identity = x  # 입력 저장

        # 다운샘플링 필요 시
        if self.downsample is not None:
            identity = self.downsample(x)

        # Residual 블록의 주요 계산
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        # 스킵 연결
        out += identity
        out = self.relu(out)
        return out


In [6]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=2):
        super(ResNet, self).__init__()
        self.in_channels = 64

        # 초기 컨볼루션 레이어
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Residual Blocks
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        # 마지막 분류 레이어
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


In [7]:
# ResNet18은 각 스테이지에서 [2, 2, 2, 2] 블록을 사용합니다.
def ResNet18(num_classes=2):
    return ResNet(ResidualBlock, [2, 2, 2, 2], num_classes=num_classes)

# 모델 초기화
model = ResNet18(num_classes=2)  # healthy와 disease 구분
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

# 모델 확인
print(model)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=Tr

5. 손실 함수 및 최적화 기법 설정

In [8]:
criterion = nn.CrossEntropyLoss()  # 손실 함수
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 옵티마이저


6. 모델 학습

In [9]:
epochs = 30
for epoch in range(epochs):
    model.train()  # 학습 모드
    train_loss = 0.0

    # Training
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation
    model.eval()  # 평가 모드
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    val_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss/len(train_loader):.4f}, "
          f"Validation Loss: {val_loss/len(val_loader):.4f}, "
          f"Validation Accuracy: {val_accuracy:.2f}%")


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (127401984 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch [1/30], Train Loss: 1.3709, Validation Loss: 2.6726, Validation Accuracy: 70.97%
Epoch [2/30], Train Loss: 0.3803, Validation Loss: 0.7756, Validation Accuracy: 72.58%
Epoch [3/30], Train Loss: 0.3508, Validation Loss: 0.3720, Validation Accuracy: 80.65%
Epoch [4/30], Train Loss: 0.3033, Validation Loss: 0.2626, Validation Accuracy: 91.94%
Epoch [5/30], Train Loss: 0.3335, Validation Loss: 0.2879, Validation Accuracy: 88.71%
Epoch [6/30], Train Loss: 0.3411, Validation Loss: 0.2650, Validation Accuracy: 85.48%
Epoch [7/30], Train Loss: 0.2495, Validation Loss: 0.2442, Validation Accuracy: 91.94%
Epoch [8/30], Train Loss: 0.2457, Validation Loss: 0.2284, Validation Accuracy: 90.32%
Epoch [9/30], Train Loss: 0.2544, Validation Loss: 0.2116, Validation Accuracy: 88.71%
Epoch [10/30], Train Loss: 0.2647, Validation Loss: 0.2309, Validation Accuracy: 90.32%
Epoch [11/30], Train Loss: 0.2431, Validation Loss: 0.2633, Validation Accuracy: 90.32%
Epoch [12/30], Train Loss: 0.2606, Valida

7. 모델 평가

In [11]:
# 테스트 루프
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 84.42%


8. 학습된 모델 저장

In [17]:
torch.save(model.state_dict(), '/content/drive/My Drive/leaf_disease_model.pth')
print("Model saved!")


Model saved!


9. 추론 테스트

In [18]:
from PIL import Image

# 테스트 이미지 불러오기
test_image_path = "/content/leaf_data/test_leaf.jpg"
test_image = Image.open(test_image_path)

# 전처리 적용
test_image = transform(test_image).unsqueeze(0).to(device)

# 예측
model.eval()
output = model(test_image)
_, predicted = torch.max(output, 1)

classes = ['healthy', 'disease']
print(f"Predicted class: {classes[predicted.item()]}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/leaf_data/test_leaf.jpg'